# Reading data

In [24]:
import pandas as pd
df = pd.read_csv(r"C:\Users\rajes\Downloads\Flipkart Mobile - 2.csv")
df.head()

,brand,model,base_color,processor,screen_size,ROM,RAM,display_size,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Apple,iPhone SE,Black,Water,Very Small,64,2,4.7,1,1,1800,4.5,38645,32999,0.17,127.52
1,Apple,iPhone 12 Mini,Red,Ceramic,Small,64,4,5.4,2,1,2815,4.5,244,57149,0.04,1.39
2,Apple,iPhone SE,Red,Water,Very Small,64,2,4.7,1,1,1800,4.5,38645,32999,0.17,127.52
3,Apple,iPhone XR,Others,iOS,Medium,64,3,6.1,1,1,2942,4.6,5366,42999,0.10,23.07
4,Apple,iPhone 12,Red,Ceramic,Medium,128,4,6.1,2,1,2815,4.6,745,69149,0.02,5.15


In [25]:
# df.isnull().sum()
# df.duplicated()

In [26]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Lebel Encoding

In [27]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in ['brand', 'model', 'base_color', 'processor', 'screen_size']:
    le = LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le
    df[column] = le.transform(df[column])

print(df.head(3))

   brand  model  base_color  processor  screen_size  ROM  RAM  display_size  \
0      0    117           0          5            4   64    2           4.7   
1      0    114           8          0            2   64    4           5.4   
2      0    117           8          5            4   64    2           4.7   

   num_rear_camera  num_front_camera  battery_capacity  ratings  \
0                1                 1              1800      4.5   
1                2                 1              2815      4.5   
2                1                 1              1800      4.5   

   num_of_ratings  sales_price  discount_percent   sales  
0           38645        32999              0.17  127.52  
1             244        57149              0.04    1.39  
2           38645        32999              0.17  127.52  


# Model Selection & Standard Scaler

In [28]:
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

X = df.iloc[:,:-1]
y  = df.iloc[:,-1]

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scaled  = scaler.fit_transform(X_train)
X_test_scaled   = scaler.transform(X_test)


# Hyper Parameter tuning - GridSearchCV & CV - KFold

In [29]:

from sklearn.model_selection import GridSearchCV


kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Parameter grid for GradientBoostingRegressor
gbr_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gbr = GradientBoostingRegressor()
gbr_grid_search = GridSearchCV(estimator=gbr, param_grid=gbr_param_grid, cv=5, scoring='neg_mean_squared_error')
gbr_grid_search.fit(X_train_scaled, y_train)

best_gbr = gbr_grid_search.best_estimator_

print(best_gbr)

# Parameter grid for Lssso

lasso_param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-4, 1e-3, 1e-2]
}

lasso = Lasso()
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_grid_search.fit(X_train_scaled, y_train)

best_lasso = lasso_grid_search.best_estimator_

print(best_lasso)

c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GradientBoostingRegressor(learning_rate=0.2, max_depth=7, min_samples_leaf=2,
                          min_samples_split=10, n_estimators=300)
Lasso()


# Stacking the Lasso & Gbr 

In [30]:
from sklearn.ensemble import StackingRegressor

stacking_model = StackingRegressor(
    estimators=[('gbr', best_gbr), ('lasso', best_lasso)],
    final_estimator=Lasso()  
)

stacking_model.fit(X_train_scaled, y_train)

stacking_param_grid = {
    'final_estimator__alpha': [0.1, 1.0, 10.0],
    'final_estimator__max_iter': [1000, 2000, 3000]
}

stacking_grid_search = GridSearchCV(estimator=stacking_model, param_grid=stacking_param_grid, cv=5, scoring='neg_mean_squared_error')
stacking_grid_search.fit(X_train_scaled, y_train)

best_stacking_model = stacking_grid_search.best_estimator_


In [31]:
best_stacking_model

StackingRegressor(estimators=[('gbr',
                               GradientBoostingRegressor(learning_rate=0.2,
                                                         max_depth=7,
                                                         min_samples_leaf=2,
                                                         min_samples_split=10,
                                                         n_estimators=300)),
                              ('lasso', Lasso())],
                  final_estimator=Lasso(max_iter=2000))

# Cross Validation Score

In [32]:
from sklearn.model_selection import cross_val_score,KFold


best_stacking_model = stacking_grid_search.best_estimator_

best_stacking_model.fit(X_train_scaled, y_train)


val_score = cross_val_score(best_stacking_model,X_train_scaled,y_train, cv = 5)

print(val_score)
print('Mean :',val_score.mean())

[0.92403397 0.92460005 0.96198182 0.95988582 0.9405949 ]
Mean : 0.9422193107520125


In [33]:
from sklearn.metrics import mean_squared_error, r2_score

y_train_pred = best_stacking_model.predict(X_train_scaled)
y_test_pred = best_stacking_model.predict(X_test_scaled)


mse_train = mean_squared_error(y_train, y_train_pred)

r2_train = r2_score(y_train, y_train_pred)




mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Training Mean Squared Error: {mse_train:.4f}")

print(f"Training R-squared: {r2_train:.4f}")

print(f"Testing Mean Squared Error: {mse_test:.4f}")

print(f"Testing R-squared: {r2_test:.4f}")


Training Mean Squared Error: 70.3847
Training R-squared: 0.9764
Testing Mean Squared Error: 384.1504
Testing R-squared: 0.9122


# Predicting with New data

In [34]:
new_data = pd.DataFrame({
    'brand': ['Apple'],
    'model': ['iPhone 16'],
    'base_color': ['White'],
    'processor': ['A17 Bionic'],
    'screen_size': ['Medium'],
    'ROM': [128],
    'RAM': [6],
    'display_size': [6.1],
    'num_rear_camera': [2],
    'num_front_camera': [1],
    'battery_capacity': [4100],
    'ratings': [4.7],
    'num_of_ratings': [12000],
    'sales_price': [79999],
    'discount_percent': [0.1]
})


for column in ['brand', 'model', 'base_color', 'processor', 'screen_size']:
    if column in label_encoders:
        le = label_encoders[column]
        new_data[column] = new_data[column].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)


new_data_scaled = scaler.transform(new_data)

new_predictions = best_stacking_model.predict(new_data_scaled)

print("Predictions for the new data :", new_predictions)

Predictions for the new data : [22.04927359]
